# 🪧 Double Pendulum: Modes and Resonance
*A tutorial on small oscillations and normal modes of a planar double pendulum.*

**Created by Kareem Ali — 2025**


## How to run this notebook

Install required packages:
```bash
pip install sympy numpy matplotlib
```

Open this notebook in Jupyter and run cells in order.


## Introduction

This notebook derives and analyzes the small-oscillation modes of a planar double pendulum. It uses Lagrangian mechanics to obtain the equations of motion, linearizes the system for small angles, and computes the natural frequencies and mode shapes. Explanations precede each code block to help learning.


# Double Pendulum Resonance Analysis (SymPy)
A step-by-step Jupyter notebook deriving and computing the normal-mode frequencies of a double pendulum.


### Abstract


This notebook derives the equations of motion of a planar double pendulum using the Lagrangian method, linearizes them for small oscillations, and computes the normal-mode (resonance) frequencies analytically using SymPy. Explanatory notes precede each code block. A numerical example and plots visualize the two normal modes.


## Introduction


The double pendulum — two point masses connected by massless rigid rods — is a classic system in mechanics. For large amplitudes it exhibits rich and often chaotic behavior, but for **small oscillations** it behaves like two coupled harmonic oscillators. This notebook focuses on that small-angle regime: we derive the Lagrangian, obtain equations of motion, linearize about the stable equilibrium, form the generalized eigenvalue problem, and compute the natural frequencies and mode shapes using SymPy.


## 1. Setup and symbolic variables

This cell imports the required libraries and declares symbolic variables and time-dependent angle functions.

In [ ]:
# Imports and symbolic setup
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, Function, diff, Matrix, simplify, Eq, solve, factor, sqrt
sp.init_printing(use_unicode=True, pretty_print=True)

# Symbolic variables
t = sp.symbols('t', real=True)
m1, m2, l1, l2, g = symbols('m1 m2 l1 l2 g', positive=True)
theta1 = Function('theta1')(t)
theta2 = Function('theta2')(t)

print('Symbolic variables and functions declared: m1,m2,l1,l2,g and theta1(t), theta2(t)')

## 2. Geometry and Lagrangian

Define Cartesian coordinates of each mass in terms of the angles, compute velocities, kinetic and potential energies, and form the Lagrangian $L=T-V$.

In [ ]:
# Positions (pivot at origin); y positive downward
x1 = l1*sp.sin(theta1)
y1 = -l1*sp.cos(theta1)
x2 = x1 + l2*sp.sin(theta2)
y2 = y1 - l2*sp.cos(theta2)

# Velocities (time derivatives)
dx1 = diff(x1, t); dy1 = diff(y1, t)
dx2 = diff(x2, t); dy2 = diff(y2, t)

# Kinetic and potential energy
T = sp.Rational(1,2)*m1*(dx1**2 + dy1**2) + sp.Rational(1,2)*m2*(dx2**2 + dy2**2)
V = m1*g*y1 + m2*g*y2
L = sp.simplify(T - V)

# Show L briefly (simplified form)
L_s = sp.simplify(L)
print('Lagrangian constructed (simplified form shown):')
L_s


## 3. Euler–Lagrange equations (nonlinear)

Derive the full nonlinear equations of motion symbolically. These expressions are typically long; we will compute them but not expand fully in the notebook output to keep things readable.

In [ ]:
# Euler-Lagrange equations
dL_dth1dot = diff(L, diff(theta1, t))
dL_dth2dot = diff(L, diff(theta2, t))
eq1 = sp.simplify(diff(dL_dth1dot, t) - diff(L, theta1))
eq2 = sp.simplify(diff(dL_dth2dot, t) - diff(L, theta2))

print('Derived symbolic nonlinear equations of motion (stored in eq1 and eq2).')
print('Length of eq1 (characters):', len(str(eq1)))
print('Length of eq2 (characters):', len(str(eq2)))


## 4. Small-angle linearization

Apply the small-angle approximation (sin θ ≈ θ, cos θ ≈ 1). Then collect coefficients of θ¨ and θ to form the mass (M) and stiffness (K) matrices of the linear system M θ¨ + K θ = 0.

In [ ]:
# Small-angle linearization: substitute sin(theta) -> theta, cos(theta) -> 1
subs_small = {sp.sin(theta1): theta1, sp.sin(theta2): theta2, sp.cos(theta1): 1, sp.cos(theta2): 1}
eq1_lin = sp.simplify(sp.expand(eq1.subs(subs_small)))
eq2_lin = sp.simplify(sp.expand(eq2.subs(subs_small)))

# Define second derivatives
th1_dd = diff(theta1, t, 2); th2_dd = diff(theta2, t, 2)

# Extract coefficients for inertia (M) and stiffness (K)
m11 = simplify(eq1_lin.coeff(th1_dd, 1))
m12 = simplify(eq1_lin.coeff(th2_dd, 1))
m21 = simplify(eq2_lin.coeff(th1_dd, 1))
m22 = simplify(eq2_lin.coeff(th2_dd, 1))

k11 = simplify(eq1_lin.coeff(theta1, 1))
k12 = simplify(eq1_lin.coeff(theta2, 1))
k21 = simplify(eq2_lin.coeff(theta1, 1))
k22 = simplify(eq2_lin.coeff(theta2, 1))

M = Matrix([[m11, m12],[m21, m22]])
K = Matrix([[k11, k12],[k21, k22]])

M_s = simplify(M); K_s = simplify(K)

print('Mass matrix M (simplified):'); M_s
print('\nStiffness matrix K (simplified):'); K_s


## 5. Normal modes and resonance frequencies

Solve the generalized eigenvalue problem det(K - ω² M) = 0 for ω², then ω = sqrt(ω²). Compute mode shapes (nullspace) for each eigenvalue.

In [ ]:
# Generalized eigenproblem: det(K - x M) = 0 with x = omega^2
x = sp.symbols('x', real=True)
charpoly = simplify((K_s - x*M_s).det())
charpoly_fact = factor(charpoly)

sol_x = sp.solve(sp.Eq(charpoly, 0), x)  # solutions for omega^2
omega_syms = [simplify(sp.sqrt(si)) for si in sol_x]

print('Characteristic polynomial (factored):'); charpoly_fact
print('\nSolutions for omega (symbolic):')
for w in omega_syms:
    sp.pretty_print(simplify(w))

# mode vectors (symbolic)
mode_vectors = []
for si in sol_x:
    mat = simplify(K_s - si*M_s)
    null = mat.nullspace()
    if null:
        v = simplify(null[0])
        # normalize so the first component is 1 if possible
        if v[0] != 0:
            v_norm = simplify(v / v[0])
        else:
            v_norm = v
        mode_vectors.append(v_norm)
    else:
        mode_vectors.append(Matrix([0,0]))

print('\nMode shape vectors (symbolic, normalized if possible):')
mode_vectors


## 6. Numerical example and plots

Assign m1=m2=l1=l2=1 and g=9.81. Compute numeric ω and mode shapes, then plot the two normal modes as static configurations.

In [ ]:
# Numeric parameters
numeric_params = {m1:1.0, m2:1.0, l1:1.0, l2:1.0, g:9.81}

# Numeric matrices
M_num = sp.N(M_s.subs(numeric_params)); K_num = sp.N(K_s.subs(numeric_params))
x_num = [sp.N(si.subs(numeric_params)) for si in sol_x]
omega_num = [float(sp.sqrt(si)) for si in x_num]

print('Numeric omega (rad/s):', omega_num)
print('\nNumeric M and K:'); print(M_num); print(K_num)

# Numeric mode vectors
modes_num = []
for si in sol_x:
    mat = sp.N((K_s - si*M_s).subs(numeric_params))
    null = mat.nullspace()
    if null:
        v = np.array([float(null[0][0]), float(null[0][1])])
        v = v / np.linalg.norm(v)
    else:
        v = np.array([1.0, 0.0])
    modes_num.append(v)

# Plotting helper: static visualization of a mode shape
import matplotlib.pyplot as plt
def plot_mode(mode_vec, title='Mode shape', amplitude=0.4, savepath=None):
    th1 = amplitude * mode_vec[0]
    th2 = amplitude * mode_vec[1]
    l1v = float(l1.subs(numeric_params)); l2v = float(l2.subs(numeric_params))
    x1 = l1v * np.sin(th1); y1 = -l1v * np.cos(th1)
    x2 = x1 + l2v * np.sin(th2); y2 = y1 - l2v * np.cos(th2)
    plt.figure(figsize=(4,4))
    plt.plot([0, x1], [0, y1], marker='o', linewidth=3)
    plt.plot([x1, x2], [y1, y2], marker='o', linewidth=3)
    plt.gca().set_aspect('equal', adjustable='box')
    lim = 1.2 * (l1v + l2v)
    plt.xlim(-lim, lim); plt.ylim(-lim, 0.5*lim)
    plt.title(title); plt.grid(True)
    if savepath:
        plt.savefig(savepath, dpi=150, bbox_inches='tight')
    plt.show()

# Plot both modes
for i, v in enumerate(modes_num):
    plot_mode(v, title=f'Mode {i+1}: omega = {omega_num[i]:.4f} rad/s')


## 7. Discussion & Conclusion


The two normal-mode frequencies correspond physically to:

- **Lower-frequency mode:** masses swing together (in-phase), producing a softer effective restoring torque.
- **Higher-frequency mode:** masses swing in opposition (out-of-phase), producing a stiffer restoring action and higher frequency.

The symbolic expressions show explicitly how frequencies depend on masses and lengths. You can experiment by changing the numeric parameters in the previous cell.


### Appendix: full script used

The following code reproduces the computations programmatically; you can run it as a script if desired.

In [ ]:
# Appendix script: same steps as above, but packaged as a script (for reference)
# -- omitted here to keep notebook concise; see exported project files for full script.
print('Appendix placeholder - full script available as separate file.')

## Conclusion

Linearized equations, mass and stiffness matrices, and the two fundamental normal modes were derived and visualized. Try varying masses and lengths or adding damping to explore richer dynamics.
